# Load the fricking data

In [2]:

import numpy as np
import pandas as pd
from scipy.optimize import minimize

# Load new dataset
file_path = "Returns_Data_Cleaned.csv"  # Updated file path
df = pd.read_csv(file_path)

# Rename the first column (date column) properly
df.rename(columns={'Unnamed: 0': 'Dates'}, inplace=True)

# Strip spaces from column names
df.columns = df.columns.str.strip()

# Convert Date column to DateTime format
df = df[df["Dates"].astype(str).str.match(r"^\d{6}$")]  # Keep only YYYYMM format
df["Dates"] = pd.to_datetime(df["Dates"], format="%Y%m")

# Filter data for the required date range
df = df[(df["Dates"] >= "2019-12-01") & (df["Dates"] <= "2024-11-01")]

# Convert returns from percentage format to decimal (divide by 100)
df.iloc[:, 1:] = df.iloc[:, 1:].div(100)

# Set Date as index
df.set_index("Dates", inplace=True)


In [3]:

expected_returns = df.mean() # get historical expected returns
cov_matrix = df.cov() # get historical covariance matrix
#cov_matrix = cov_matrix.pop("Dates")

rf = 0.004  # define monthly risk-free rate


# Using Minimize (no short-sell)

In [4]:

# using minimze, no short-selling

# Initial equal weights
num_assets = len(expected_returns)
init_weights = np.ones(num_assets) / num_assets

# Constraints: sum of weights = 1
constraints = ({'type': 'eq', 'fun': lambda w: np.sum(w) - 1})

def sharpe_ratio(weights, mean_returns, cov_matrix, rf):
    port_return = np.dot(weights, mean_returns) - rf
    port_volatility = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
    return -port_return / port_volatility

# Bounds: No short-selling (weights between 0 and 1)
bounds = [(0, 1) for _ in range(num_assets)]

# Solve the constrained optimization
result_long = minimize(sharpe_ratio, init_weights, args=(expected_returns, cov_matrix, rf), 
                       method='SLSQP', constraints=constraints, bounds=bounds)

# Extract long-only optimal weights
optimal_weights_long = result_long.x

# Select top 5 assets (highest weights)
top_5_indices_long = np.argsort(optimal_weights_long)[-5:]
selected_assets_long = df.columns[top_5_indices_long]

# Extract new sub-data
sub_returns_long = df[selected_assets_long]
sub_expected_returns_long = expected_returns.iloc[top_5_indices_long]
sub_cov_matrix_long = cov_matrix.iloc[top_5_indices_long, top_5_indices_long]

# Re-optimize for the top 5 assets
init_weights_5_long = np.ones(5) / 5
result_5_long = minimize(sharpe_ratio, init_weights_5_long, args=(sub_expected_returns_long, sub_cov_matrix_long, rf), 
                         method='SLSQP', constraints={'type': 'eq', 'fun': lambda w: np.sum(w) - 1}, bounds=[(0, 1)] * 5)

optimal_weights_5_long = result_5_long.x

print("\nTop 5 Assets (no-short selling):")
for asset, weight in zip(selected_assets_long, optimal_weights_5_long):
    print(f"{asset}: {weight:.4f}")



Top 5 Assets (no-short selling):
Soda: 0.0000
Agric: 0.0435
Guns: 0.1402
Coal: 0.2177
Chips: 0.5986


# Quadratic Programming (no short-sell)

In [5]:
# quadratic programming to solve for no-short selling case
import cvxpy as cp

def max_sharpe_qp(mean_returns, cov_matrix, rf):
    num_assets = len(mean_returns)
    
    # Define optimization variables
    w = cp.Variable(num_assets)
    
    # Portfolio return and volatility
    port_return = (mean_returns - rf) @ w
    port_volatility = cp.sqrt(cp.quad_form(w, cov_matrix))
    
    # Sharpe ratio maximization (Equivalent to minimizing -SR)
    objective = cp.Maximize(port_return / port_volatility)
    
    # Constraints
    constraints = [cp.sum(w) == 1, w >= 0]
    
    # Solve QP
    prob = cp.Problem(objective, constraints)
    prob.solve()
    
    return w.value  # Optimal weights

# Compute optimal weights
optimal_weights_qp = max_sharpe_qp(expected_returns, cov_matrix, rf)

# Select top 5 assets
top_5_indices_qp = np.argsort(optimal_weights_qp)[-5:]
selected_assets_qp = df.columns[top_5_indices_qp]

# Print results
print("\nTop 5 Assets (Long-Only, QP) with Weights:")
for asset, weight in zip(selected_assets_qp, optimal_weights_qp[top_5_indices_qp]):
    print(f"{asset}: {weight:.4f}")


IndexError: tuple index out of range

# Analytical Markowitz solution (short-sell)

In [13]:
# with short-selling, using analytical solution

def max_sharpe_markowitz(mean_returns, cov_matrix, rf):
    excess_returns = mean_returns - rf
    inv_cov = np.linalg.inv(cov_matrix)
    
    # Optimal weights (analytical solution)
    w_star = inv_cov @ excess_returns
    w_star /= np.sum(w_star)  # Normalize to sum to 1
    
    return w_star

# Compute optimal weights (unconstrained)
optimal_weights_mv = max_sharpe_markowitz(expected_returns, cov_matrix, rf)

# Select top 5 assets
top_5_indices_mv = np.argsort(np.abs(optimal_weights_mv))[-5:]
selected_assets_mv = df.columns[top_5_indices_mv]

# Print results
print("\nTop 5 Assets (Unconstrained, Markowitz) with Weights:")
for asset, weight in zip(selected_assets_mv, optimal_weights_mv[top_5_indices_mv]):
    print(f"{asset}: {weight:.4f}")



Top 5 Assets (Unconstrained, Markowitz) with Weights:
Trans: -0.8355
Insur: 0.8463
Util: 0.8929
Chems: -1.3089
Mach: 1.6783


In [12]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize

# Assume expected_returns and cov_matrix are already defined
rf = 0.02 / 12  # Risk-free rate (monthly)

# Function: Analytical Markowitz solution (with short selling allowed)
def max_sharpe_markowitz(mean_returns, cov_matrix, rf):
    excess_returns = mean_returns - rf
    inv_cov = np.linalg.inv(cov_matrix)
    
    # Optimal weights (analytical solution)
    w_star = inv_cov @ excess_returns
    w_star /= np.sum(w_star)  # Normalize to sum to 1
    
    return w_star

# Compute optimal weights (unconstrained)
optimal_weights_mv = max_sharpe_markowitz(expected_returns, cov_matrix, rf)

# Select top 5 assets (highest absolute weights)
top_5_indices_mv = np.argsort(np.abs(optimal_weights_mv))[-5:]
selected_assets_mv = df.columns[top_5_indices_mv]
selected_weights_mv = optimal_weights_mv[top_5_indices_mv]

# Create DataFrame for Short-Selling Allowed
df_short_selling = pd.DataFrame({
    "Industry": selected_assets_mv,
    "Weights": selected_weights_mv
})

# --- NO SHORT-SELLING CASE ---

# Initial equal weights
num_assets = len(expected_returns)
init_weights = np.ones(num_assets) / num_assets

# Constraints: sum of weights = 1
constraints = ({'type': 'eq', 'fun': lambda w: np.sum(w) - 1})

def sharpe_ratio(weights, mean_returns, cov_matrix, rf):
    port_return = np.dot(weights, mean_returns) - rf
    port_volatility = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
    return -port_return / port_volatility

# Bounds: No short-selling (weights between 0 and 1)
bounds = [(0, 1) for _ in range(num_assets)]

# Solve the constrained optimization
result_long = minimize(sharpe_ratio, init_weights, args=(expected_returns, cov_matrix, rf), 
                       method='SLSQP', constraints=constraints, bounds=bounds)

# Extract long-only optimal weights
optimal_weights_long = result_long.x

# Select top 5 assets (highest weights)
top_5_indices_long = np.argsort(optimal_weights_long)[-5:]
selected_assets_long = df.columns[top_5_indices_long]
selected_weights_long = optimal_weights_long[top_5_indices_long]

# Create DataFrame for No Short-Selling Constraint
df_no_short_selling = pd.DataFrame({
    "Industry (Short-Sale Constraint)": selected_assets_long,
    "Weights (Constraint)": selected_weights_long
})

# Display results
print("\nTop 5 Assets (Short-Selling Allowed):")
print(df_short_selling)

print("\nTop 5 Assets (No Short-Selling):")
print(df_no_short_selling)



Top 5 Assets (Short-Selling Allowed):
  Industry   Weights
0    Trans -0.835505
1    Insur  0.846251
2     Util  0.892855
3    Chems -1.308899
4     Mach  1.678349

Top 5 Assets (No Short-Selling):
  Industry (Short-Sale Constraint)  Weights (Constraint)
0                            Smoke              0.001312
1                            Agric              0.075095
2                             Guns              0.182899
3                             Coal              0.191897
4                            Chips              0.548797
